## Домашнее задание №5. «Гибридные рекомендатльные системы»
 - Датасет ml-latest
- Вспомнить подходы, которые мы разбирали
- Выбрать понравившийся подход к гибридным системам
- Написать свою

In [2]:
from surprise import SVD, SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

import pandas as pd
import numpy as np

In [3]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [4]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [5]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [121]:
num_movies = movies_with_ratings.movieId.unique().shape[0]
uniques = movies_with_ratings.movieId.unique()

In [112]:
user_vector = {}

for user, group in movies_with_ratings.groupby('userId'):
    user_vector[user] = np.zeros(num_movies)
    
    for i in range(len(group.movieId.values)):
        m = np.argwhere(uniques==group.movieId.values[i])[0][0]
        r = group.rating.values[i]
        user_vector[user][int(m - 1)] = r

In [138]:
user_vector[1.0][9723]

4.0

In [141]:
group.rating

214       5.0
534       5.0
954       4.5
1678      4.5
2309      5.0
2582      4.0
2913      4.0
3178      3.5
3623      4.5
3727      5.0
3773      4.0
4436      3.0
4544      3.5
5520      5.0
5852      5.0
6324      4.0
7103      5.0
7856      5.0
8166      5.0
8290      3.0
8968      3.0
8988      3.5
9056      3.5
9184      3.0
9642      3.0
10347     3.0
10866     3.5
11289     5.0
11495     5.0
11592     5.0
         ... 
99667     3.0
99728     4.0
99762     4.0
99775     3.5
99791     3.5
99812     5.0
99830     4.5
99853     4.0
99886     5.0
99892     3.5
99908     3.5
99912     3.0
99934     3.0
99968     3.0
99974     2.5
99987     4.5
100001    3.0
100009    3.0
100049    4.0
100055    4.0
100068    3.5
100114    3.5
100115    3.5
100142    5.0
100244    4.0
100250    4.0
100312    5.0
100327    5.0
100352    5.0
100429    3.0
Name: rating, Length: 1302, dtype: float64

In [135]:
uniques[9723]

193609

In [136]:
movies_with_ratings[movies_with_ratings.movieId == 193609]

,movieId,title,genres,userId,rating,timestamp
100853,193609,Andrew Dice Clay: Dice Rules (1991),Comedy,331.0,4.0,1.537158e+09


In [6]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [7]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [9]:
trainset, testset = train_test_split(data, test_size=.15, random_state=42)

In [36]:
algo = SVDpp(n_factors=20, n_epochs=20)
algo.fit(trainset)

In [19]:
test_pred = algo.test(testset)

In [20]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8697


0.8697164341369683

In [32]:
algo.predict(uid=2.0, iid='Tommy Boy (1995)').est

2.58673955947146

In [35]:
movies_with_ratings[movies_with_ratings.userId == current_user_id]

,movieId,title,genres,userId,rating,timestamp
8652,318,"Shawshank Redemption, The (1994)",Crime|Drama,2.0,3.0,1.445715e+09
9186,333,Tommy Boy (1995),Comedy,2.0,4.0,1.445715e+09
34763,1704,Good Will Hunting (1997),Drama|Romance,2.0,4.5,1.445715e+09
55521,3578,Gladiator (2000),Action|Adventure|Drama,2.0,4.0,1.445715e+09
72623,6874,Kill Bill: Vol. 1 (2003),Action|Crime|Thriller,2.0,4.0,1.445715e+09
76960,8798,Collateral (2004),Action|Crime|Drama|Thriller,2.0,3.5,1.445715e+09
82957,46970,Talladega Nights: The Ballad of Ricky Bobby (2...,Action|Comedy,2.0,4.0,1.445715e+09
83525,48516,"Departed, The (2006)",Crime|Drama|Thriller,2.0,4.0,1.445715e+09
86885,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,2.0,4.5,1.445715e+09
87792,60756,Step Brothers (2008),Comedy,2.0,5.0,1.445715e+09


In [33]:
current_user_id = 2.0
user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()

scores = []
titles = []

for movie in movies_with_ratings.title.unique():
    if movie in user_movies:
        continue
        
    scores.append(algo.predict(uid=current_user_id, iid=movie).est)
    titles.append(movie)

In [34]:
sorted(scores)[-10:]

[4.292689858000501,
 4.295808661909504,
 4.318442322513884,
 4.337828767593964,
 4.337843408833879,
 4.340747640570685,
 4.370101942593006,
 4.378359466080809,
 4.41525065895355,
 4.427056416302067]